<a href="https://colab.research.google.com/github/BrendaLoznik/waterpumps/blob/main/Waterpumps.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

competition: https://www.drivendata.org/competitions/7/pump-it-up-data-mining-the-water-table/

geographical data: https://heartbeat.fritz.ai/working-with-geospatial-data-in-machine-learning-ad4097c7228d

optional data quality report: https://github.com/benjimr/Data-Summarizer-for-a-Data-Quality-Report/blob/master/Reporter.py

data quality report: https://github.com/ColinLeverger/machine-learning-python

In [1]:
!nvidia-smi

Sat May 15 11:00:53 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   66C    P8    11W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

###**1.1 Loading Libraries**

In [2]:
#basic libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from scipy import stats

###**1.2 Loading data**

In [3]:
from google.colab import drive 
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
train_labels = pd.read_csv('/content/gdrive/MyDrive/Kaggle/Waterpumps/train_labels.csv')
train_values = pd.read_csv('/content/gdrive/MyDrive/Kaggle/Waterpumps/train_values.csv', parse_dates = ['date_recorded'])
test_values = pd.read_csv('/content/gdrive/MyDrive/Kaggle/Waterpumps/test_values.csv', parse_dates = ['date_recorded'])

In [5]:
#merge train values and train labels to single dataset
train= pd.merge(train_labels , train_values, on='id')


In [6]:
#concatenate train and test for easier data minipulation
raw = pd.concat([train, test_values])
raw.head()

,id,status_group,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
0,69572,functional,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,Lake Nyasa,Mnyusi B,Iringa,11,5,Ludewa,Mundindi,109,True,GeoData Consultants Ltd,VWC,Roman,False,1999,gravity,gravity,gravity,vwc,user-group,pay annually,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
1,8776,functional,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,Lake Victoria,Nyamara,Mara,20,2,Serengeti,Natta,280,NaN,GeoData Consultants Ltd,Other,NaN,True,2010,gravity,gravity,gravity,wug,user-group,never pay,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
2,34310,functional,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,Pangani,Majengo,Manyara,21,4,Simanjiro,Ngorika,250,True,GeoData Consultants Ltd,VWC,Nyumba ya mungu pipe scheme,True,2009,gravity,gravity,gravity,vwc,user-group,pay per bucket,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe
3,67743,non functional,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,Ruvuma / Southern Coast,Mahakamani,Mtwara,90,63,Nanyumbu,Nanyumbu,58,True,GeoData Consultants Ltd,VWC,NaN,True,1986,submersible,submersible,submersible,vwc,user-group,never pay,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe
4,19728,functional,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,Lake Victoria,Kyanyamisa,Kagera,18,1,Karagwe,Nyakasimbi,0,True,GeoData Consultants Ltd,NaN,NaN,True,0,gravity,gravity,gravity,other,other,never pay,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe


In [7]:
#convert faulty datatypes
raw[['region_code','district_code', 'num_private']] = raw[['region_code','district_code', 'num_private']].astype('object')

In [8]:
#divide features into numerical and cateogorical for ease of describtion
numerical = ['id', 'amount_tsh', 'date_recorded', 'gps_height',  'longitude', 'latitude', 
             'population', 'construction_year' ]

categorical = ['status_group', 'funder', 'installer','wpt_name','num_private',  'basin', 'subvillage', 'region', 'region_code',
               'district_code', 'lga', 'ward', 'public_meeting', 'recorded_by', 'scheme_management', 
               'scheme_name', 'permit', 'extraction_type', 'extraction_type_group','extraction_type_class',
               'management', 'management_group', 'payment','payment_type', 'water_quality', 'quality_group', 'quantity',
               'quantity_group', 'source', 'source_type', 'source_class', 'waterpoint_type', 'waterpoint_type_group']


print("Number of numerical features: {}".format(len(numerical)))
print("Number of categorical features: {}".format(len(categorical)))


Number of numerical features: 8
Number of categorical features: 33


###**2.1 Exploring the data**

####**2.1.1 Numerical data**

In [ ]:
raw[numerical].head()

,id,amount_tsh,date_recorded,gps_height,longitude,latitude,population,construction_year
0,69572,6000.0,2011-03-14,1390,34.938093,-9.856322,109,1999
1,8776,0.0,2013-03-06,1399,34.698766,-2.147466,280,2010
2,34310,25.0,2013-02-25,686,37.460664,-3.821329,250,2009
3,67743,0.0,2013-01-28,263,38.486161,-11.155298,58,1986
4,19728,0.0,2011-07-13,0,31.130847,-1.825359,0,0


In [ ]:
#raw[numerical].describe()

In [ ]:
#print percentage of missing numerical datapoints
#round(raw[numerical].isna().sum()*100/raw[numerical].isna().count()).sort_values(ascending=False)

In [ ]:
#explore the cardinality of numerical features
#for column in raw[numerical]:
  #unique = raw[column].nunique()
  #print(column,":", unique)
 

###Data Quality report Numerical features

In [ ]:
data_types = pd.DataFrame(
    raw[numerical].dtypes,
    columns=['Data Type'])

missing_data = pd.DataFrame(
    raw[numerical].isnull().sum(),
    columns=['Missing Values'])

unique_values = pd.DataFrame(
    columns=['Unique Values']
)
for row in list(raw[numerical].columns.values):
    unique_values.loc[row] = [raw[numerical][row].nunique()]

maximum_values = pd.DataFrame(
    columns=['Maximum Value'])
for row in list(raw[numerical].columns.values):
    maximum_values.loc[row] = [raw[numerical][row].max()]

minimum_values = pd.DataFrame(
    columns=['Minimum Value'])
for row in list(raw[numerical].columns.values):
    minimum_values.loc[row] = [raw[numerical][row].min()]

count_values = pd.DataFrame(
    columns=['Count Value'])
for row in list(raw[numerical].columns.values):
    count_values.loc[row] = [raw[numerical][row].count()]

mean_values = pd.DataFrame(
    columns=['Mean Value'])
for row in list(raw[numerical].columns.values):
    mean_values.loc[row] = [raw[numerical][row].mean()]

quartile_1 = pd.DataFrame(
    columns=['1st Quartile'])
for row in list(raw[numerical].columns.values):
    quartile_1.loc[row] = [raw[numerical][row].quantile(0.25)]

median = pd.DataFrame(
    columns=['Median'])
for row in list(raw[numerical].columns.values):
    median.loc[row] = [raw[numerical][row].quantile(0.5)]

quartile_3 = pd.DataFrame(
    columns=['3rd Quartile'])
for row in list(raw[numerical].columns.values):
    quartile_3.loc[row] = [raw[numerical][row].quantile(0.75)]

dq_report = data_types.join(count_values).join(missing_data).join(unique_values).join(minimum_values).join(maximum_values).join(mean_values).join(quartile_1).join(median).join(quartile_3)
dq_report['Missing %'] = (dq_report['Missing Values'] / len(raw[numerical]) *100)
dq_report

,Data Type,Count Value,Missing Values,Unique Values,Minimum Value,Maximum Value,Mean Value,1st Quartile,Median,3rd Quartile,Missing %
id,int64,74250,0,74250,0,74249,37124.5,18562.2,37124.5,55686.8,0.0
amount_tsh,float64,74250,0,102,0,350000,318.686,0,0,20,0.0
date_recorded,datetime64[ns],74250,0,369,2001-03-26 00:00:00,2013-12-03 00:00:00,2012-03-28 09:51:49.527267072,2011-04-01 00:00:00,2012-10-10 00:00:00,2013-02-09 00:00:00,0.0
gps_height,int64,74250,0,2456,-90,2777,665.667,0,364,1317,0.0
longitude,float64,74250,0,71870,0,40.3452,34.0743,33.0868,34.9075,37.1817,0.0
latitude,float64,74250,0,71869,-11.6494,-2e-08,-5.70177,-8.52567,-5.02654,-3.32506,0.0
population,int64,74250,0,1128,0,30500,180.751,0,25,215,0.0
construction_year,int64,74250,0,55,0,2013,1298.46,0,1986,2004,0.0


Sat May 15 10:57:42 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   67C    P0    71W / 149W |    124MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
#plot selected feature
ax = sns.histplot(x=raw['amount_tsh'])
ax.set (title = "test", ylabel = "Vzl mean score")
plt.show()

In [1]:
# plot distribution of features
fig, axes = plt.subplots(nrows=2,ncols=4, figsize = (15,8))
sns.histplot(x=raw['amount_tsh'],  ax=axes[0][0],  color = 'green' )
sns.histplot(x=raw['amount_tsh'],  ax=axes[0][1], color = 'green')
sns.histplot(x=raw['amount_tsh'],  ax=axes[0][2], color = 'green')
sns.histplot(x=raw['amount_tsh'],  ax=axes[0][3], color = 'green')

#sns.kdeplot(x=df['Ingr_Haver_1'],  ax=axes[1][0],  color = 'green' )
#sns.kdeplot(x=df['Ingr_Palmpitvetzuur_1'],  ax=axes[1][1], color = 'green')
#sns.kdeplot(x=df['K_1'],  ax=axes[1][2], color = 'green')
#sns.kdeplot(x=df['RE_1'],  ax=axes[1][3], color = 'green')


#axes[0][0].set(xlabel = "Temperature", ylabel='Count', title="Effect of Temperature on Count")
#axes[0][1].set(xlabel='Humidity', ylabel='Count',title="Effect of Humidity on Count")
#axes[1][0].set(xlabel='windspeed', ylabel='Count',title="Effect of Windspeed on Count")
#axes[1][1].set(xlabel='Windspeed_mice', ylabel='Count',title="Effect of Windspeed(Mice) on Count")
plt.show()

NameError: ignored

In [120]:
feature = test.columns.tolist()

def graphs(feature, rows, columns):
    fig, axes = plt.subplots(nrows=rows,ncols=columns, figsize = (20,20))
    plot = 0
    for i in range(rows):
        for j in range(columns):
             ax = sns.kdeplot(x=test[feature[plot]],  ax = axes[i][j],  color = 'green' )
             ax.set_title("Skeweness: {}".format(test[feature[plot]].skew()))
             plot +=1

graphs(feature, 3, 5)

####**2.1.1 Categorical data**

In [ ]:
raw[categorical].head()

,status_group,funder,installer,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,public_meeting,recorded_by,scheme_management,scheme_name,permit,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
0,functional,Roman,Roman,none,0,Lake Nyasa,Mnyusi B,Iringa,11,5,Ludewa,Mundindi,True,GeoData Consultants Ltd,VWC,Roman,False,gravity,gravity,gravity,vwc,user-group,pay annually,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
1,functional,Grumeti,GRUMETI,Zahanati,0,Lake Victoria,Nyamara,Mara,20,2,Serengeti,Natta,NaN,GeoData Consultants Ltd,Other,NaN,True,gravity,gravity,gravity,wug,user-group,never pay,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
2,functional,Lottery Club,World vision,Kwa Mahundi,0,Pangani,Majengo,Manyara,21,4,Simanjiro,Ngorika,True,GeoData Consultants Ltd,VWC,Nyumba ya mungu pipe scheme,True,gravity,gravity,gravity,vwc,user-group,pay per bucket,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe
3,non functional,Unicef,UNICEF,Zahanati Ya Nanyumbu,0,Ruvuma / Southern Coast,Mahakamani,Mtwara,90,63,Nanyumbu,Nanyumbu,True,GeoData Consultants Ltd,VWC,NaN,True,submersible,submersible,submersible,vwc,user-group,never pay,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe
4,functional,Action In A,Artisan,Shuleni,0,Lake Victoria,Kyanyamisa,Kagera,18,1,Karagwe,Nyakasimbi,True,GeoData Consultants Ltd,NaN,NaN,True,gravity,gravity,gravity,other,other,never pay,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe


In [ ]:
# print absolute number  of missing continious data points
#print(raw[categorical].isna().sum().sort_values(ascending=False))

In [ ]:
#print percentage of missing numerical datapoints
#round(raw[categorical].isna().sum()*100/raw[categorical].isna().count()).sort_values(ascending=False)

In [ ]:
#print percentage of missing numerical datapoints
#round(raw[categorical].isna().sum()*100/raw[categorical].isna().count()).sort_values(ascending=False)

In [ ]:
#value = raw['status_group'].value_counts()
#value

In [ ]:
#explore the cardinality of numerical features
#for column in raw[categorical]:
  #mode = raw[column].value_counts()
  #print(mode)
 

In [ ]:
#how many unique gps coordinates are there?
#temp = raw.copy()
#temp['location'] = temp['longitude'].astype(str) + '-' + temp['latitude'].astype(str)
#temp['location'].nunique()


In [118]:
data_types = pd.DataFrame(
    raw[categorical].dtypes,
    columns=['Data Type'])

count_values = pd.DataFrame(
    columns=['Count Value'])
for row in list(raw[categorical].columns.values):
    count_values.loc[row] = [raw[categorical][row].count()]

missing_data = pd.DataFrame(
    raw[categorical].isnull().sum(),
    columns=['Missing Values'])

unique_values = pd.DataFrame(
    columns=['Unique Values'])
for row in list(raw[categorical].columns.values):
    unique_values.loc[row] = [raw[categorical][row].nunique()]

mode_values = pd.DataFrame(
    columns=['Mode'])
for row in list(raw[categorical].columns.values):
    mode_values.loc[row] = [raw[categorical][row].mode()[0]]
    mode = mode_values.loc[row]

#mode frequency
listModeFreq = []
for row in categorical:
    mode = raw[row].mode().iat[0]
    ModeFreq = raw[row].value_counts()[mode]
    #print(x, mode, raw[x].value_counts()[mode])
    listModeFreq .append(ModeFreq )
listModeFreq  = np.array(listModeFreq )

dq_report = data_types.join(count_values).join(missing_data).join(unique_values).join(mode_values)
dq_report['Mode freq.'] = listModeFreq 
dq_report['Mode %'] = (dq_report['Mode freq.'] / dq_report['Count Value']*100) 
dq_report['Missing %'] = (dq_report['Missing Values'] / len(raw[categorical]) *100)
dq_report  

,Data Type,Count Value,Missing Values,Unique Values,Mode,Mode freq.,Mode %,Missing %
status_group,object,59400,14850,3,functional,32259,54.3081,20.000000
funder,object,69746,4504,2140,Government Of Tanzania,11299,16.2002,6.065993
installer,object,69718,4532,2410,DWE,21751,31.1985,6.103704
wpt_name,object,74250,0,45684,none,4440,5.9798,0.000000
num_private,object,74250,0,68,0,73299,98.7192,0.000000
basin,object,74250,0,9,Lake Victoria,12871,17.3347,0.000000
subvillage,object,73780,470,21425,Shuleni,646,0.875576,0.632997
region,object,74250,0,21,Iringa,6599,8.88754,0.000000
region_code,object,74250,0,27,11,6608,8.89966,0.000000
district_code,object,74250,0,20,1,15299,20.6047,0.000000


,Data Type,Count Value,Missing Values,Unique Values,Mode,Mode freq.,Mode %,Missing %
status_group,object,59400,14850,3,functional,32259,54.3081,20.000000
funder,object,69746,4504,2140,Government Of Tanzania,11299,16.2002,6.065993
installer,object,69718,4532,2410,DWE,21751,31.1985,6.103704
wpt_name,object,74250,0,45684,none,4440,5.9798,0.000000
num_private,object,74250,0,68,0,73299,98.7192,0.000000
basin,object,74250,0,9,Lake Victoria,12871,17.3347,0.000000
subvillage,object,73780,470,21425,Shuleni,646,0.875576,0.632997
region,object,74250,0,21,Iringa,6599,8.88754,0.000000
region_code,object,74250,0,27,11,6608,8.89966,0.000000
district_code,object,74250,0,20,1,15299,20.6047,0.000000


In [ ]:
def graphs(feature, rows, columns):
    fig, axes = plt.subplots(nrows=rows,ncols=columns, figsize = (20,20))
    plot = 0
    for i in range(rows):
        for j in range(columns):
             ax = sns.kdeplot(x=test[feature[plot]],  ax = axes[i][j],  color = 'green' )
             ax.set_title("Skeweness: {}".format(test[feature[plot]].skew()))
             plot +=1

In [ ]:
graphs(feature, 3, 5)